In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from tqdm import tqdm
from functools import partial
import copy
import ot

from typing import List, Optional, Tuple
import hydra
from hydra import initialize, compose
import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf

import sys
import os, shutil
import time
import datetime

In [ ]:
# Импорт пользовательских функций
sys.path.append('..')
device = 'cpu'


# Load the autoreload extension
%load_ext autoreload
# Set autoreload mode
%autoreload 2

from src.ScoreNetwork import ScoreNetwork
from src.DSBM_model_mod_v3 import DSBM, train_dsbm, train
from src.generate_gaussian_cloud import generate_gaussian_cloud
from src.test_fn import test_fn

## Загрузка и задание параметров эксперимента

In [ ]:
# Импорт параметров модели и данных

with initialize(version_base=None, config_path="../configurations"):
    
    cfg: DictConfig = compose(config_name="gaussian_MOD_v3.yaml")

if cfg.get("seed"):
    pl.seed_everything(cfg.seed, workers=True)
print(cfg)

RESULT_DIR = '../results_v3/' + cfg.paths.experiments_dir_name

# Вариант 1: Явная проверка с пользовательской ошибкой
if os.path.exists(RESULT_DIR):
    print(f"\n\nДиректория {RESULT_DIR} уже существует. Сгенерирована новая директория с индексом")
    now = datetime.datetime.now()
    timestamp = now.strftime("%Y%m%d_%H%M%S")
    RESULT_DIR = f'{RESULT_DIR}_{timestamp}'
    
os.makedirs(RESULT_DIR, exist_ok=True)

RESULT_DIR = RESULT_DIR + '/'

OmegaConf.save(cfg, RESULT_DIR + 'config.yaml')

## Генерация train-test датасетов (двух гауссиан)

In [ ]:
# Генерация датасетов

# train
x0 = generate_gaussian_cloud(
    vars_diag=cfg.x0_vars_diag,  
    cov_pairs=cfg.x0_cov_pairs, 
    mean=cfg.x0_mean, 
    dataset_size=cfg.dataset_size, 
    plot=True
)  

x1 = generate_gaussian_cloud(
    vars_diag=cfg.x1_vars_diag,  
    cov_pairs=cfg.x1_cov_pairs, 
    mean=cfg.x1_mean, 
    dataset_size=cfg.dataset_size, 
    plot=True
)  


# test
x0_test = generate_gaussian_cloud(
    vars_diag=cfg.x0_vars_diag,  
    cov_pairs=cfg.x0_cov_pairs, 
    mean=cfg.x0_mean, 
    dataset_size=cfg.test_dataset_size, 
    plot=False
)  

x1_test = generate_gaussian_cloud(
    vars_diag=cfg.x1_vars_diag,  
    cov_pairs=cfg.x1_cov_pairs, 
    mean=cfg.x1_mean, 
    dataset_size=cfg.test_dataset_size, 
    plot=False
)  


# Приведение к требуемому типу
x0 = torch.tensor(x0).to(torch.float32).to(device)
x1 = torch.tensor(x1).to(torch.float32).to(device)
x0_test = torch.tensor(x0_test).to(torch.float32).to(device)
x1_test = torch.tensor(x1_test).to(torch.float32).to(device)

x_pairs = torch.stack([x0, x1], dim=1).to(device)
x_pairs_test = torch.stack([x0_test, x1_test], dim=1).to(device)


print('Гауссиана 0:')
print('mean: ', x0.mean().item())
print('var: ', x0.var().item())
print('cnt: ', x0.shape[0])
print('dim: ', x0.shape[1])

print('\nГауссиана 1:')
print('mean: ', x1.mean().item())
print('var: ', x1.var().item())
print('cnt: ', x1.shape[0])
print('dim: ', x1.shape[1])

## Инициализация модели

In [ ]:
# Определение структуры модели

activation_fn = hydra.utils.get_class(cfg.activation_fn)()
hidden_size = cfg.net_hidden_layer_width
dim = cfg.dim
net_fn = partial(ScoreNetwork, input_dim=dim+1, layer_widths=[hidden_size, hidden_size, dim], activation_fn=activation_fn)  

num_steps = cfg.num_steps
sigma = cfg.sigma
inner_iters = cfg.inner_iters
batch_size = cfg.batch_size


model = DSBM(net_fwd=net_fn().to(device), 
             net_bwd=net_fn().to(device), 
             num_steps=num_steps, sig=sigma, first_coupling=cfg.first_coupling)
train_fn = train_dsbm

n_parameters = sum(p.numel() for p in model.net_fwd.parameters() if p.requires_grad)
print(f"Number of parameters: <{n_parameters}>")

## Preprocessing

In [ ]:
# Нормализация данных

all_data = torch.stack([x0, x1], dim=1).to(device) # Формируем полный пулл данных, которые учитываем для обучения скалера
model.normalizer.fit(all_data)
model.normalizer_fitted = True # Флаг для модели, что задействован скалер
model.sig = sigma * model.normalizer.A # Масштабирем sigma
x0_norm= model.normalizer.normalize(x0.clone())
x1_norm = model.normalizer.normalize(x1.clone())
x_pairs = torch.stack([x0_norm, x1_norm], dim=1).to(device) # Формируем обучающую пару

print('Гауссиана 0:')
print('mean: ', x0_norm.mean().item())
print('var: ', x0_norm.var().item())
print('cnt: ', x0_norm.shape[0])
print('dim: ', x0_norm.shape[1])

print('\nГауссиана 1:')
print('mean: ', x1_norm.mean().item())
print('var: ', x1_norm.var().item())
print('cnt: ', x1_norm.shape[0])
print('dim: ', x1_norm.shape[1])

In [ ]:
logs = dict()
logs['optimal_result_dict'] = {'mean': x0_test.mean(0).mean(0).item(), 'var': x0_test.var(0).mean(0).item(), 'cov': (np.sqrt(5) - 1) / 2}
logs['time_list'] = []
logs['time_list_res'] = []
logs['result_list'] = {k: [] for k in logs['optimal_result_dict'].keys()}

## Train-test loops

In [ ]:
# train-test
model, logs = train(cfg, model, x_pairs, x_pairs_test, logs, 20, 1e-5, RESULT_DIR)

In [ ]:
#val
df_SWD = test_fn(cfg, model, N_samples = 10)
df_SWD

# Сохранение результатов

In [ ]:
# Сохраняем результаты
# Сохраняем последнюю версию модели
torch.save(model.state_dict(), RESULT_DIR + 'best_model.pt')

df_result = pd.DataFrame(logs['result_list'])
df_result.to_csv(RESULT_DIR + 'df_result.csv')
df_result.to_pickle(RESULT_DIR+ 'df_result.pkl')

df_time = pd.DataFrame(logs['time_list'])
df_time.to_csv(RESULT_DIR + 'df_time.csv')
df_time.to_pickle(RESULT_DIR+ 'df_time.pkl')

df_SWD.to_csv(RESULT_DIR + 'df_wasserstein_distance.csv')